## Point pro 

In [7]:
!git clone https://github.com/apple/ml-depth-pro.git
%cd ml-depth-pro

# 2. Install the package
!pip install -e .

# 3. Download the pretrained models
!bash get_pretrained_models.sh

Cloning into 'ml-depth-pro'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 45 (delta 7), reused 2 (delta 2), pack-reused 21 (from 1)
Receiving objects: 100% (45/45), 2.50 MiB | 9.81 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/Users/hendrik/.Trash/ml-depth-pro/ml-depth-pro/ml-depth-pro
Obtaining file:///Users/hendrik/.Trash/ml-depth-pro/ml-depth-pro/ml-depth-pro
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for depth_pro (pyproject.toml) ... done
  Created wheel for depth_pro: filename=depth_pro-0.1-0.editable-py3-none-any.whl size=4825 sha256=68672ec1bd914f568c4b16640d7fa41b3659f3db3460dffa8554c5232ab83ece
  Stored in directory: /private/var/folders/1g/rwnkwjjs0w9_xcc0cwsf40dw0000gn/T/pip-ephem-whee

#### Single image to point cloud with estimated Focal length

In [8]:
import sys
sys.path.append('./ml-depth-pro')
from src.depth_pro.depth_pro import *
from src.depth_pro.utils import *


import numpy as np
import torch
import matplotlib.pyplot as plt
import open3d as o3d
import os
import cv2
from PIL import Image

def setup_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU (CUDA) is available and will be used")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("GPU (MPS) is available and will be used")
    else:
        device = torch.device("cpu")
        print("GPU is not available, using CPU")
    return device

def get_image_paths_from_directory(directory, extensions=(".jpg", ".jpeg", ".png")):
    """Fetch all image file paths from the given directory."""
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.lower().endswith(extensions)]

def load_and_process_images_from_directory(directory, transform, device):
    """Load and preprocess all images from a directory."""
    image_paths = get_image_paths_from_directory(directory)
    
    images_tensors = []
    focal_lengths = []
    
    for image_path in image_paths:
        image, _, f_px = depth_pro.load_rgb(image_path)
        images_tensors.append(transform(image).to(device))
        focal_lengths.append(f_px)
    
    return image_paths, images_tensors, focal_lengths

def run_depth_inference(model, image_tensors, focal_lengths):
    """Run depth inference on multiple images."""
    depth_maps = []
    focal_px_values = []
    
    with torch.no_grad():
        for image_tensor, f_px in zip(image_tensors, focal_lengths):
            prediction = model.infer(image_tensor, f_px=f_px)
            depth_maps.append(prediction["depth"].cpu().numpy())
            focal_px_values.append(prediction["focallength_px"].item())
    
    return depth_maps, focal_px_values

def create_point_clouds(depth_maps, images, focal_px_values, scale_ratio=1.0):
    """Create multiple point clouds from depth maps and images."""
    point_clouds = []
    
    for depth_map, image, focal_px in zip(depth_maps, images, focal_px_values):
        H, W = depth_map.shape
        image = ensure_image_format(image)
        
        x_grid, y_grid = np.meshgrid(np.arange(W), np.arange(H))
        cx, cy = W / 2, H / 2

        Z = depth_map * scale_ratio
        X = (x_grid - cx) * Z / focal_px
        Y = (y_grid - cy) * Z / focal_px

        point_map = np.stack((X, Y, Z), axis=-1)

        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(point_map.reshape(-1, 3))
        point_cloud.colors = o3d.utility.Vector3dVector(image.reshape(-1, 3))

        valid_points = Z.flatten() > 0
        point_cloud.points = o3d.utility.Vector3dVector(np.asarray(point_cloud.points)[valid_points])
        point_cloud.colors = o3d.utility.Vector3dVector(np.asarray(point_cloud.colors)[valid_points])

        point_clouds.append(point_cloud)
    
    return point_clouds

def ensure_image_format(image):
    """Ensure image is in the correct format (H, W, 3) and properly scaled."""
    if isinstance(image, torch.Tensor):
        image = image.cpu().detach().numpy()
    
    if image.shape[0] == 3:  # If in (3, H, W) format
        image = np.transpose(image, (1, 2, 0))
    
    if image.min() < 0 or image.max() > 1:
        image = (image + 1) / 2  # Normalize to [0,1]
    
    return np.clip(image, 0, 1)

def visualize_results(images, depth_maps):
    """Visualize multiple images and depth maps."""
    num_images = len(images)
    fig, axes = plt.subplots(num_images, 2, figsize=(10, 5 * num_images))
    
    if num_images == 1:
        axes = [axes]  # Make it iterable for a single image
    
    for i, (image, depth_map) in enumerate(zip(images, depth_maps)):
        axes[i][0].imshow(image)
        axes[i][0].set_title(f"Original Image {i+1}")
        axes[i][0].axis('off')

        depth_plot = axes[i][1].imshow(depth_map, cmap='viridis')
        axes[i][1].set_title(f"Depth Map {i+1}")
        fig.colorbar(depth_plot, ax=axes[i][1], label='Depth (m)')
        axes[i][1].axis('off')

    plt.tight_layout()
    plt.show()

def save_point_clouds(point_clouds, image_paths):
    """Save multiple point clouds to 'generated_pointclouds/'."""
    output_dir = "generated_pointclouds"
    os.makedirs(output_dir, exist_ok=True)  # Ensure directory exists

    for point_cloud, image_path in zip(point_clouds, image_paths):
        filename = os.path.basename(image_path).split('.')[0] + ".ply"
        output_path = os.path.join(output_dir, filename)
        o3d.io.write_point_cloud(output_path, point_cloud)
        print(f"Saved: {output_path}")

def calculate_distances(depth_maps, points_of_interest=None):
    """
    Calculate absolute metric distances from the camera to objects in multiple depth maps.
    
    Parameters:
    depth_maps (list of np.array): List of depth maps generated by Depth Pro.
    points_of_interest (list of lists): Optional. List of lists, where each sublist contains
                                        (y, x) coordinates of specific points to measure.

    Returns:
    dict: A dictionary containing distance statistics or point-specific distances for each depth map.
    """
    results = {}
    
    for i, depth_map in enumerate(depth_maps):
        if points_of_interest is None:
            valid_depths = depth_map[depth_map > 0]  # Ignore zero or negative depths
            results[f'image_{i+1}'] = {
                'min_distance': np.min(valid_depths),
                'max_distance': np.max(valid_depths),
                'mean_distance': np.mean(valid_depths)
            }
        else:
            distances = {}
            for j, (y, x) in enumerate(points_of_interest[i]):
                distance = depth_map[y, x]
                distances[f'point_{j}'] = distance if distance > 0 else None
            results[f'image_{i+1}'] = distances
    
    return results



if __name__ == "__main__":

    DEFAULT_MONODEPTH_CONFIG_DICT = DepthProConfig(
    patch_encoder_preset="dinov2l16_384",
    image_encoder_preset="dinov2l16_384",
    checkpoint_uri= "ml-depth-pro/checkpoints/depth_pro.pt",
    decoder_features=256,
    use_fov_head=True,
    fov_encoder_preset="dinov2l16_384",
    )
    # 1. Setup Device
    device = setup_device()

    # --- Assumptions: These functions exist in your depth_pro module ---
    # Load the depth estimation model
    model, transform = create_model_and_transforms(config = DEFAULT_MONODEPTH_CONFIG_DICT)

    model = model.to(device)
    model.eval()
    # Load the necessary image transforms    # --- End Assumptions ---

    # 2. Define Image Paths
    image_paths = [
        # "/Users/hendrik/OMGrab/output_grid.jpg",
        # Add the right image if you have it, e.g.:
        "/Users/hendrik/OMGrab/Stereo images/right_motorcycle.jpg"
    ]
    checkpoint_path = "./ml-depth-pro/checkpoints/depth_pro.pt"

    if not all(os.path.exists(p) for p in image_paths):
        print("Error: One or more specified image paths do not exist.")
        exit()

    # 3. Load and Preprocess Images
    print("Loading and preprocessing images...")
    images_tensors = []
    focal_lengths = []
    original_images_for_vis_and_pc = [] # Keep original images for visualization/point cloud

    for image_path in image_paths:
        # Load image and focal length using depth_pro function
        image_np, _, f_px = load_rgb(image_path)
        original_images_for_vis_and_pc.append(image_np) # Store original numpy image

        # Apply transform and move to device
        image_tensor = transform(image_np).to(device)
        images_tensors.append(image_tensor)
        focal_lengths.append(f_px)
    print(f"Loaded {len(images_tensors)} images.")

    # 4. Run Depth Inference
    print("Running depth inference...")
    depth_maps, focal_px_values = run_depth_inference(model, images_tensors, focal_lengths)
    print("Depth inference complete.")

    # 5. Create Point Clouds
    print("Creating point clouds...")
    # Use the original numpy images loaded earlier for coloring
    point_clouds = create_point_clouds(depth_maps, original_images_for_vis_and_pc, focal_px_values)
    print(f"Created {len(point_clouds)} point clouds.")

    # 6. Save Point Clouds
    print("Saving point clouds...")
    save_point_clouds(point_clouds, image_paths)
    print("Point clouds saved.")

    # 7. Optional: Visualize Results
    # Ensure images are in the correct format for matplotlib (e.g., numpy array [0,1])
    vis_images = [ensure_image_format(img) for img in original_images_for_vis_and_pc]
    visualize_results(vis_images, depth_maps)
    print("Visualization displayed.")

GPU (MPS) is available and will be used


FileNotFoundError: [Errno 2] No such file or directory: 'ml-depth-pro/checkpoints/depth_pro.pt'

#### Point cloud from video with custom focal length

In [5]:
import sys
sys.path.append('./ml-depth-pro')
from src.depth_pro.depth_pro import *
from src.depth_pro.utils import *
import numpy as np
import torch
import matplotlib.pyplot as plt
import open3d as o3d
import os
import cv2
from PIL import Image
import glob # Needed for finding saved files
import time  # Optional: for timing

# --- Function Definitions (Assuming setup_device, create_point_clouds, etc. are here) ---
def setup_device():
    # ... (implementation from previous versions) ...
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("GPU (MPS) is available and will be used")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU (CUDA) is available and will be used")
    else:
        device = torch.device("cpu")
        print("GPU is not available, using CPU")
    return device

def create_point_clouds(depth_maps, images, focal_px_values, scale_ratio=1.0):
    # ... (implementation from previous versions - check if it needs cx, cy based on new H, W) ...
    point_clouds = []
    for depth_map, image, focal_px in zip(depth_maps, images, focal_px_values):
        # image should be the resized image (H, W, 3) numpy array
        # depth_map should correspond to the resized image dimensions
        H, W = depth_map.shape # Get dimensions from depth map (should match resized image)

        # Ensure image is numpy (it should be already after cv2.resize and cvtColor)
        if isinstance(image, torch.Tensor):
           image = image.cpu().numpy() # Fallback
        if image.shape[0] != H or image.shape[1] != W:
            print(f"Warning: Mismatch between depth map ({H}x{W}) and image ({image.shape[0]}x{image.shape[1]}) dimensions in create_point_clouds. Resizing image.")
            image = cv2.resize(image, (W, H)) # Resize color image to match depth map if necessary

        # Normalize image colors if they are not 0-1
        if image.max() > 1.0:
             image = image.astype(np.float32) / 255.0
        image = np.clip(image, 0, 1)


        # Calculate cx, cy based on the *actual* dimensions being used
        cx, cy = W / 2, H / 2

        x_grid, y_grid = np.meshgrid(np.arange(W), np.arange(H))

        Z = depth_map * scale_ratio
        # Need to handle division by zero if focal_px is 0 or Z is 0
        # Avoid division by zero for X, Y calculation where Z=0
        valid_depth_mask = Z > 1e-6 # Small epsilon to avoid division issues
        X = np.zeros_like(Z)
        Y = np.zeros_like(Z)

        X[valid_depth_mask] = (x_grid[valid_depth_mask] - cx) * Z[valid_depth_mask] / focal_px
        Y[valid_depth_mask] = (y_grid[valid_depth_mask] - cy) * Z[valid_depth_mask] / focal_px


        point_map = np.stack((X, Y, Z), axis=-1)

        point_cloud = o3d.geometry.PointCloud()
        # Reshape carefully, ensure image colors match points
        points_vec = point_map.reshape(-1, 3)
        colors_vec = image.reshape(-1, 3)

        # Filter points based on the valid depth mask used earlier
        valid_points_flat = valid_depth_mask.flatten()
        point_cloud.points = o3d.utility.Vector3dVector(points_vec[valid_points_flat])
        point_cloud.colors = o3d.utility.Vector3dVector(colors_vec[valid_points_flat])

        # Original Z > 0 check - redundant if valid_depth_mask is used correctly, but safe to keep
        # valid_points = Z.flatten() > 0 # Z is already scaled
        # point_cloud.points = o3d.utility.Vector3dVector(np.asarray(point_cloud.points)[valid_points])
        # point_cloud.colors = o3d.utility.Vector3dVector(np.asarray(point_cloud.colors)[valid_points])

        point_clouds.append(point_cloud)
    return point_clouds
# ... (other functions if needed) ...


if __name__ == "__main__":
    # --- Configuration ---
    INPUT_VIDEO_PATH = "depth maps/Video_1_L_rectified_src.mp4"
    FRAME_INTERVAL = 5 # Process one frame every FRAME_INTERVAL frames
    TARGET_HEIGHT = 480 # Target height for processing
    TARGET_WIDTH = 854 # Target width for processing

    POINT_CLOUD_SAVE_DIR = f"generated_pointclouds/frames_{TARGET_HEIGHT}p" # Add resolution to folder name
    OUTPUT_VIDEO_FILENAME = f"point_cloud_video_{TARGET_HEIGHT}p.mp4" # Add resolution to filename
    OUTPUT_VIDEO_DIR = "generated_pointclouds" # Where the final video goes

    POINT_CLOUD_SCALE_RATIO = 1.0 # Should be 1.0 if f_px is provided to infer
    # Output video dimensions should match the processing dimensions
    VIS_WIDTH = TARGET_WIDTH
    VIS_HEIGHT = TARGET_HEIGHT
    FPS = 10 # Frame rate of the output video

    # Ensure output directories exist
    os.makedirs(POINT_CLOUD_SAVE_DIR, exist_ok=True)
    os.makedirs(OUTPUT_VIDEO_DIR, exist_ok=True)

    print(f"Input Video: {INPUT_VIDEO_PATH}")
    print(f"Processing Resolution: {TARGET_WIDTH}x{TARGET_HEIGHT}")
    print(f"Individual Point Clouds Save Dir: {POINT_CLOUD_SAVE_DIR}")
    print(f"Output Video: {os.path.join(OUTPUT_VIDEO_DIR, OUTPUT_VIDEO_FILENAME)} ({VIS_WIDTH}x{VIS_HEIGHT} @ {FPS}fps)")
    print(f"Processing every {FRAME_INTERVAL} frames.")

    # Define Camera Intrinsics - NOTE: These are for the ORIGINAL resolution.
    # If resizing, intrinsics need to be scaled accordingly if used precisely.
    # However, we are passing the average f_px to model.infer, which the model might
    # handle internally or override. The create_point_clouds function uses the average f_px.
    # Scaling f_px and c_x, c_y might be needed for perfect geometric accuracy
    # if create_point_clouds were using cx, cy directly from a scaled matrix.
    camera_intrinsics_orig = np.array([
        [1.06936178e+03, 0.00000000e+00, 5.55299273e+02],
        [0.00000000e+00, 1.06969222e+03, 9.44910636e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]
    ])
    fx_orig = camera_intrinsics_orig[0, 0]
    fy_orig = camera_intrinsics_orig[1, 1]
    # Calculate an average focal length. If the model adjusts based on input aspect ratio,
    # this might be sufficient. For precise geometry, scaling fx/fy separately is better.
    focal_length_orig_px = (fx_orig + fy_orig) / 2.0
    print(f"Using original average focal length: {focal_length_orig_px:.2f} px (Note: Frames will be resized)")
    # We will pass this focal length value to model.infer and create_point_clouds.
    # The model might interpret this f_px relative to the input image size/aspect ratio.

    # --- Model and Device Setup ---
    # ... (model loading remains the same) ...
    DEFAULT_MONODEPTH_CONFIG_DICT = DepthProConfig( # Use src.depth_pro
        patch_encoder_preset="dinov2l16_384",
        image_encoder_preset="dinov2l16_384",
        checkpoint_uri="ml-depth-pro/checkpoints/depth_pro.pt",
        decoder_features=256,
        use_fov_head=True,
        fov_encoder_preset="dinov2l16_384",
    )
    device = setup_device()
    model, transform = create_model_and_transforms(config=DEFAULT_MONODEPTH_CONFIG_DICT) # Use src.depth_pro
    model = model.to(device)
    model.eval()
    print("Model loaded and moved to device.")


    # ===========================================================
    # Phase 1: Generate and Save Individual Point Clouds (with resizing)
    # ===========================================================
    print("\n--- Starting Phase 1: Generating and Saving Point Clouds ---")
    start_time_phase1 = time.time()

    # ... (video file checks remain the same) ...
    if not os.path.exists(INPUT_VIDEO_PATH):
        print(f"Error: Video file not found at {INPUT_VIDEO_PATH}")
        exit()
    cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
    print(f"number of frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")
    if not cap.isOpened():
        print(f"Error: Could not open video file {INPUT_VIDEO_PATH}")
        exit()


    frame_count = 0
    processed_frame_count = 0
    saved_files_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break # End of video

        if frame_count % FRAME_INTERVAL == 0:
            print(f"Processing frame {frame_count} for point cloud saving...")
            processed_frame_count += 1

            # <<< Resize Frame >>>
            frame_resized = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT), interpolation=cv2.INTER_AREA)

            # 1. Preprocess *Resized* Frame
            frame_rgb_np = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            image_tensor = transform(frame_rgb_np).to(device)
            # Pass the *original* average focal length. The model's `infer` might adapt.
            f_px_tensor = torch.tensor(focal_length_orig_px, dtype=torch.float32, device=device)

            # 2. Run Depth Inference
            with torch.no_grad():
                # Model's infer function should handle resizing internally and
                # return depth map corresponding to the input tensor's size (resized frame)
                prediction = model.infer(image_tensor, f_px=f_px_tensor)
                depth_map = prediction["depth"].squeeze().cpu().numpy()

            # Check if depth map dimensions match target size
            if depth_map.shape[0] != TARGET_HEIGHT or depth_map.shape[1] != TARGET_WIDTH:
                 print(f"Warning: Depth map size {depth_map.shape} doesn't match target {TARGET_HEIGHT}x{TARGET_WIDTH}. Resizing depth map.")
                 depth_map = cv2.resize(depth_map, (TARGET_WIDTH, TARGET_HEIGHT), interpolation=cv2.INTER_NEAREST) # Use nearest for depth


            # 3. Create Point Cloud for this frame (using resized image and corresponding depth)
            frame_point_clouds = create_point_clouds(
                [depth_map],          # Depth map corresponding to resized frame
                [frame_rgb_np],       # Resized RGB frame (as numpy)
                [focal_length_orig_px], # Original average focal length (for scaling)
                scale_ratio=POINT_CLOUD_SCALE_RATIO
            )

            # 4. Save the Point Cloud if valid
            # ... (saving logic remains the same) ...
            if frame_point_clouds and frame_point_clouds[0].has_points():
                pcd = frame_point_clouds[0]
                if pcd.has_points():
                    filename = f"frame_{frame_count:05d}.ply"
                    output_path = os.path.join(POINT_CLOUD_SAVE_DIR, filename)
                    o3d.io.write_point_cloud(output_path, pcd)
                    saved_files_count += 1
                else:
                     print(f"Frame {frame_count}: Point cloud empty after processing/filtering.")
            else:
                 print(f"Frame {frame_count}: No point cloud generated.")


        frame_count += 1

    # ... (rest of Phase 1 cleanup remains the same) ...
    cap.release()
    end_time_phase1 = time.time()
    print(f"--- Phase 1 Complete ({end_time_phase1 - start_time_phase1:.2f} seconds) ---")
    print(f"Processed {processed_frame_count} frames, saved {saved_files_count} point cloud files.")

    # ... (check for saved_files_count remains the same) ...


    # ===========================================================
    # Phase 2: Create Video from Saved Point Clouds
    # ===========================================================
    print("\n--- Starting Phase 2: Creating Video from Saved Point Clouds ---")
    start_time_phase2 = time.time()

    # Find all saved .ply files from the correct directory
    ply_files = sorted(glob.glob(os.path.join(POINT_CLOUD_SAVE_DIR, "frame_*.ply")))

    # ... (check if ply_files list is empty remains the same) ...
    if not ply_files:
        print(f"Error: No .ply files found in {POINT_CLOUD_SAVE_DIR}. Cannot create video.")
        exit()


    # --- Video Output Setup (using VIS_WIDTH, VIS_HEIGHT which match TARGET_WIDTH, TARGET_HEIGHT) ---
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Or 'XVID' for .avi
    output_video_path = os.path.join(OUTPUT_VIDEO_DIR, OUTPUT_VIDEO_FILENAME)
    # Use VIS_WIDTH, VIS_HEIGHT for VideoWriter size
    out_video = cv2.VideoWriter(output_video_path, fourcc, FPS, (VIS_WIDTH, VIS_HEIGHT))
    print(f"Output video will be saved to: {output_video_path}")

    # --- Open3D Visualizer Setup (using VIS_WIDTH, VIS_HEIGHT) ---
    vis = o3d.visualization.Visualizer()
    # Use VIS_WIDTH, VIS_HEIGHT for Visualizer window size
    vis.create_window(width=VIS_WIDTH, height=VIS_HEIGHT, visible=False) # Hidden window
    # ... (setting background color, getting view control remains the same) ...
    render_options = vis.get_render_option()
    render_options.background_color = np.asarray([0.0, 0.0, 0.0])
    view_control = vis.get_view_control()

    # ... (setting initial view based on first cloud remains the same) ...
    try:
        first_pcd = o3d.io.read_point_cloud(ply_files[0])
        points = np.asarray(first_pcd.points)
        if points.size > 0:
             median_z = np.median(points[:, 2])
        else:
             median_z = 5.0
    except Exception as e:
        print(f"Warning: Could not read first point cloud {ply_files[0]} to set initial view: {e}")
        median_z = 5.0
    print(f"Setting initial view: lookat=[0, 0, {median_z:.2f}], front=[0, 0, 1], up=[0, -1, 0]")
    view_control.set_front([0, 0, 1])
    view_control.set_lookat([0, 0, median_z])
    view_control.set_up([0, -1, 0])
    view_control.set_zoom(0.4) # Adjust zoom if needed


    # --- Loop through saved files and render ---
    print(f"Rendering {len(ply_files)} point clouds for video...")
    # ... (looping, reading ply, rendering, writing frame remains the same) ...
    for i, ply_file in enumerate(ply_files):
        print(f"Rendering file {i+1}/{len(ply_files)}: {os.path.basename(ply_file)}")
        try:
            pcd = o3d.io.read_point_cloud(ply_file)
            if not pcd.has_points():
                print("  ...Skipping empty point cloud.")
                frame_bgr = np.zeros((VIS_HEIGHT, VIS_WIDTH, 3), dtype=np.uint8)
            else:
                vis.clear_geometries()
                vis.add_geometry(pcd)
                vis.poll_events()
                vis.update_renderer()
                o3d_img = vis.capture_screen_float_buffer(do_render=True)
                frame_np = (np.asarray(o3d_img) * 255).astype(np.uint8)
                frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)

        except Exception as e:
            print(f"  Error reading or rendering file {ply_file}: {e}")
            frame_bgr = np.zeros((VIS_HEIGHT, VIS_WIDTH, 3), dtype=np.uint8)

        out_video.write(frame_bgr)


    # --- Cleanup ---
    # ... (releasing video, destroying windows remains the same) ...
    out_video.release()
    vis.destroy_window()
    cv2.destroyAllWindows()
    end_time_phase2 = time.time()
    print(f"--- Phase 2 Complete ({end_time_phase2 - start_time_phase2:.2f} seconds) ---")
    print(f"Output video saved to: {output_video_path}")


Input Video: depth maps/Video_1_L_rectified_src.mp4
Processing Resolution: 854x480
Individual Point Clouds Save Dir: generated_pointclouds/frames_480p
Output Video: generated_pointclouds/point_cloud_video_480p.mp4 (854x480 @ 10fps)
Processing every 5 frames.
Using original average focal length: 1069.53 px (Note: Frames will be resized)
GPU (MPS) is available and will be used


FileNotFoundError: [Errno 2] No such file or directory: 'ml-depth-pro/checkpoints/depth_pro.pt'

#### Generate video only

In [9]:
import open3d as o3d
import numpy as np
import glob
import os
import cv2
import math
import time

# --- Configuration ---
POINT_CLOUD_DIR = "/Users/hendrik/OMGrab/point_clouds"
OUTPUT_VIDEO_FILENAME = "point_cloud_video_set_lookat_method.mp4" # New filename
OUTPUT_VIDEO_DIR = "generated_pointclouds"

VIS_WIDTH = 1080
VIS_HEIGHT = 1920
FPS = 15

CAMERA_ORIGIN = np.array([0.0, -3.0, 0.0])
CAMERA_X_ROTATION_DEG = -45.0
CAMERA_Z_ROTATION_DEG = 180.0

# --- End Configuration ---

def create_video_from_point_clouds(
    pcd_dir,
    output_dir,
    output_filename,
    width,
    height,
    fps,
    camera_origin,
    camera_x_rotation_deg,
    camera_z_rotation_deg
):
    """Generates video from point clouds using set_lookat/front/up."""
    # ... (Setup code: directory checks, file finding, video writer init remains the same) ...
    os.makedirs(output_dir, exist_ok=True)
    output_video_path = os.path.join(output_dir, output_filename)
    ply_files = sorted(glob.glob(os.path.join(pcd_dir, "frame_*.ply")))
    if not ply_files:
        print(f"Error: No .ply files found in {pcd_dir} matching 'frame_*.ply'.")
        return
    print(f"Found {len(ply_files)} point cloud files in {pcd_dir}.")
    print(f"Output video: {output_video_path} ({width}x{height} @ {fps}fps)")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    if not out_video.isOpened():
        print(f"Error: Could not open video writer for path {output_video_path}")
        return

    # --- Open3D Visualizer Setup ---
    # ... (Visualizer creation, background color remains the same) ...
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=width, height=height, visible=False)
    render_options = vis.get_render_option()
    render_options.background_color = np.asarray([0.0, 0.0, 0.0])
    print("Open3D visualizer created.")
    view_control = vis.get_view_control()

    # --- Calculate Camera View Vectors ---
    # Convert degrees to radians
    theta_x = math.radians(camera_x_rotation_deg)
    theta_z = math.radians(camera_z_rotation_deg)

    # Rotation matrix for X-axis rotation (around world X)
    cos_x, sin_x = math.cos(theta_x), math.sin(theta_x)
    R_x = np.array([
        [1.0, 0.0,   0.0],
        [0.0, cos_x, -sin_x],
        [0.0, sin_x,  cos_x]
    ])

    # Rotation matrix for Z-axis rotation (around world Z)
    cos_z, sin_z = math.cos(theta_z), math.sin(theta_z)
    R_z = np.array([
        [cos_z, -sin_z, 0.0],
        [sin_z,  cos_z, 0.0],
        [0.0,    0.0,   1.0]
    ])

    # Combine rotations: Apply Z rotation first, then X rotation in world frame.
    # R = Rx * Rz -> This matrix rotates vectors from camera frame to world frame.
    R_cam_to_world = R_x @ R_z
    print("Camera orientation matrix (Cam axes in World coords):\n", R_cam_to_world)

    # Default camera view vectors in camera's own coordinate system:
    #  - points along +Z
    #  - 'up' direction corresponds to -Y
    default_front_vec = np.array([0.0, 0.0, 1.0])
    default_up_vec = np.array([0.0, -1.0, 0.0])

    # Rotate these default vectors by the camera's orientation to get them in world coordinates
    world_front_vec = R_cam_to_world @ default_front_vec
    world_up_vec = R_cam_to_world @ default_up_vec

    # The point the camera looks at is the origin plus the direction vector
    lookat_point = camera_origin + world_front_vec

    print(f"Calculated Camera Eye: {camera_origin}")
    print(f"Calculated Camera Lookat: {lookat_point}")
    print(f"Calculated Camera Front: {world_front_vec}")
    print(f"Calculated Camera Up: {world_up_vec}")


    # --- Set View using set_lookat/set_front/set_up ---
    # These methods might be more reliable for setting orientation
    view_control.set_front(world_front_vec)
    view_control.set_up(world_up_vec)
    view_control.set_lookat(lookat_point) # Center view target
    # Eye position is implicitly set based on lookat and front, but we can try setting zoom
    view_control.set_zoom(0.6) # Adjust zoom level as needed (try different values)

    print("Camera view parameters set using set_lookat/front/up.")

    # --- Loop through saved files and render ---
    # ... (The loop for reading, rendering, capturing, and writing frames remains the same) ...
    print(f"Rendering {len(ply_files)} point clouds...")
    start_time = time.time()
    for i, ply_file in enumerate(ply_files):
        if (i+1) % 50 == 0:
             print(f"  Processing file {i+1}/{len(ply_files)}: {os.path.basename(ply_file)}")
        try:
            pcd = o3d.io.read_point_cloud(ply_file)
            if not pcd.has_points():
                print(f"  Warning: Skipping empty point cloud file {os.path.basename(ply_file)}.")
                frame_bgr = np.zeros((height, width, 3), dtype=np.uint8)
            else:
                vis.clear_geometries()
                vis.add_geometry(pcd)
                # Re-apply view parameters inside loop IF NEEDED (usually not)
                # view_control.set_front(world_front_vec)
                # view_control.set_up(world_up_vec)
                # view_control.set_lookat(lookat_point)
                vis.poll_events()
                vis.update_renderer()
                o3d_img = vis.capture_screen_float_buffer(do_render=True)
                frame_np = (np.asarray(o3d_img) * 255).astype(np.uint8)
                frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"  Error reading or rendering file {ply_file}: {e}")
            frame_bgr = np.zeros((height, width, 3), dtype=np.uint8)
        out_video.write(frame_bgr)

    # --- Cleanup ---
    # ... (Releasing video writer, destroying window remains the same) ...
    end_time = time.time()
    print(f"\nRendering complete in {end_time - start_time:.2f} seconds.")
    out_video.release()
    vis.destroy_window()
    cv2.destroyAllWindows()
    print(f"Output video saved to: {output_video_path}")


# --- Main Execution ---
if __name__ == "__main__":
    create_video_from_point_clouds(
        pcd_dir=POINT_CLOUD_DIR,
        output_dir=OUTPUT_VIDEO_DIR,
        output_filename=OUTPUT_VIDEO_FILENAME,
        width=VIS_WIDTH,
        height=VIS_HEIGHT,
        fps=FPS,
        camera_origin=CAMERA_ORIGIN,
        camera_x_rotation_deg=CAMERA_X_ROTATION_DEG,
        camera_z_rotation_deg=CAMERA_Z_ROTATION_DEG
    )

Found 429 point cloud files in /Users/hendrik/OMGrab/point_clouds.
Output video: generated_pointclouds/point_cloud_video_set_lookat_method.mp4 (1080x1920 @ 15fps)
Open3D visualizer created.
Camera orientation matrix (Cam axes in World coords):
 [[-1.00000000e+00 -1.22464680e-16  0.00000000e+00]
 [ 8.65956056e-17 -7.07106781e-01  7.07106781e-01]
 [-8.65956056e-17  7.07106781e-01  7.07106781e-01]]
Calculated Camera Eye: [ 0. -3.  0.]
Calculated Camera Lookat: [ 0.         -2.29289322  0.70710678]
Calculated Camera Front: [0.         0.70710678 0.70710678]
Calculated Camera Up: [ 1.22464680e-16  7.07106781e-01 -7.07106781e-01]
Camera view parameters set using set_lookat/front/up.
Rendering 429 point clouds...
  Processing file 50/429: frame_00049.ply
  Processing file 100/429: frame_00099.ply
  Processing file 150/429: frame_00149.ply
  Processing file 200/429: frame_00199.ply
  Processing file 250/429: frame_00249.ply
  Processing file 300/429: frame_00299.ply
  Processing file 350/429: 

In [12]:
import open3d as o3d
import numpy as np
import glob
import os
import cv2
import math
import time

# --- Configuration ---
POINT_CLOUD_DIR = "/Users/hendrik/OMGrab/point_clouds"
OUTPUT_VIDEO_FILENAME = "point_cloud_video_stereo_calibrate_method.mp4" # New filename
OUTPUT_VIDEO_DIR = "generated_pointclouds"

VIS_WIDTH = 1080  # Width for visualizer and video frame
VIS_HEIGHT = 1920 # Height for visualizer and video frame
FPS = 15

# Note: Camera origin/rotation parameters are not used in this method,
# as the view is set by a fixed extrinsic matrix and point cloud rotation.
# CAMERA_ORIGIN = np.array([0.0, -3.0, 0.0])
# CAMERA_X_ROTATION_DEG = -45.0
# CAMERA_Z_ROTATION_DEG = 180.0

# --- End Configuration ---

def create_video_from_point_clouds_stereo_method( # Renamed function
    pcd_dir,
    output_dir,
    output_filename,
    width,
    height,
    fps
):
    """
    Generates video from point clouds using the method from stereo_calibrate.ipynb.
    Rotates point clouds and uses a fixed extrinsic matrix for the camera view.
    """
    os.makedirs(output_dir, exist_ok=True)
    output_video_path = os.path.join(output_dir, output_filename)
    ply_files = sorted(glob.glob(os.path.join(pcd_dir, "frame_*.ply")))

    if not ply_files:
        print(f"Error: No .ply files found in {pcd_dir} matching 'frame_*.ply'.")
        return
    print(f"Found {len(ply_files)} point cloud files in {pcd_dir}.")
    print(f"Output video: {output_video_path} ({width}x{height} @ {fps}fps)")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # Ensure VideoWriter uses the correct width and height
    out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    if not out_video.isOpened():
        print(f"Error: Could not open video writer for path {output_video_path}")
        return

    # --- Open3D Visualizer Setup ---
    vis = o3d.visualization.Visualizer()
    # Ensure visualizer window matches output video dimensions
    vis.create_window(width=width, height=height, visible=False)
    render_options = vis.get_render_option()
    render_options.background_color = np.asarray([0.0, 0.0, 0.0]) # Black background
    render_options.point_size = 1.0 # Adjust point size if needed (example used 4.0)
    print("Open3D visualizer created.")

    # --- Define Point Cloud Rotation ---
    # Rotation matrix to rotate point cloud 180 degrees around X-axis
    R_pc_fix = o3d.geometry.get_rotation_matrix_from_xyz((np.pi, 0, 0))
    print("Defined point cloud rotation matrix (180 deg around X).")

    # --- Set Camera View using Fixed Extrinsic Matrix ---
    view_control = vis.get_view_control()
    pinhole_params = view_control.convert_to_pinhole_camera_parameters()

    # Fixed extrinsic matrix from stereo_calibrate example
    # Places camera at [0,0,2] looking toward origin, Y-axis flipped in view
    fixed_extrinsic = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, -1, 0], # Adjust Z offset (last value, '2') to move camera closer/further
        [0, 0, 0, 1]
    ])
    pinhole_params.extrinsic = fixed_extrinsic
    print("Setting fixed camera extrinsic matrix:\n", fixed_extrinsic)

    # Apply the modified parameters back to the view control
    view_control.convert_from_pinhole_camera_parameters(pinhole_params, allow_arbitrary=True)

    # Adjust zoom level (smaller value = more zoom)
    zoom_level = 0.4 # Adjust as needed (example used 0.1, which is very zoomed in)
    view_control.set_zoom(zoom_level)
    print(f"Camera view parameters set. Zoom level: {zoom_level}")

    # Add first geometry to initialize renderer state (optional but good practice)
    try:
        pcd_initial = o3d.io.read_point_cloud(ply_files[0])
        pcd_initial.rotate(R_pc_fix, center=(0, 0, 0)) # Apply rotation
        vis.add_geometry(pcd_initial)
        vis.poll_events()
        vis.update_renderer()
        vis.clear_geometries() # Clear it after setup if looping starts from first file
        print("Initial visualizer state set.")
    except Exception as e:
        print(f"Warning: Could not load or process first point cloud for initial setup: {e}")


    # --- Loop through saved files and render ---
    print(f"Rendering {len(ply_files)} point clouds...")
    start_time = time.time()
    for i, ply_file in enumerate(ply_files):
        if (i+1) % 50 == 0:
             print(f"  Processing file {i+1}/{len(ply_files)}: {os.path.basename(ply_file)}")
        try:
            pcd = o3d.io.read_point_cloud(ply_file)
            if not pcd.has_points():
                print(f"  Warning: Skipping empty point cloud file {os.path.basename(ply_file)}.")
                frame_bgr = np.zeros((height, width, 3), dtype=np.uint8)
            else:
                # <<< Apply the 180-degree X-axis rotation to the point cloud >>>
                pcd.rotate(R_pc_fix, center=(0, 0, 0))

                vis.clear_geometries()
                vis.add_geometry(pcd)

                # Camera view is fixed, just update rendering state
                vis.poll_events()
                vis.update_renderer()

                # Capture frame
                o3d_img = vis.capture_screen_float_buffer(do_render=True)
                frame_np = (np.asarray(o3d_img) * 255).astype(np.uint8)
                frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)

        except Exception as e:
            print(f"  Error reading or rendering file {ply_file}: {e}")
            frame_bgr = np.zeros((height, width, 3), dtype=np.uint8)

        out_video.write(frame_bgr)

    # --- Cleanup ---
    end_time = time.time()
    print(f"\nRendering complete in {end_time - start_time:.2f} seconds.")
    out_video.release()
    vis.destroy_window()
    cv2.destroyAllWindows()
    print(f"Output video saved to: {output_video_path}")


# --- Main Execution ---
if __name__ == "__main__":
    # Call the new function
    create_video_from_point_clouds_stereo_method(
        pcd_dir=POINT_CLOUD_DIR,
        output_dir=OUTPUT_VIDEO_DIR,
        output_filename=OUTPUT_VIDEO_FILENAME,
        width=VIS_WIDTH,
        height=VIS_HEIGHT,
        fps=FPS
    )

Found 429 point cloud files in /Users/hendrik/OMGrab/point_clouds.
Output video: generated_pointclouds/point_cloud_video_stereo_calibrate_method.mp4 (1080x1920 @ 15fps)
Open3D visualizer created.
Defined point cloud rotation matrix (180 deg around X).
Setting fixed camera extrinsic matrix:
 [[ 1  0  0  0]
 [ 0 -1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  1]]
Camera view parameters set. Zoom level: 0.4
Initial visualizer state set.
Rendering 429 point clouds...
  Processing file 50/429: frame_00049.ply
  Processing file 100/429: frame_00099.ply
  Processing file 150/429: frame_00149.ply
  Processing file 200/429: frame_00199.ply
  Processing file 250/429: frame_00249.ply
  Processing file 300/429: frame_00299.ply
  Processing file 350/429: frame_00349.ply
  Processing file 400/429: frame_00399.ply

Rendering complete in 15.79 seconds.
Output video saved to: generated_pointclouds/point_cloud_video_stereo_calibrate_method.mp4


In [15]:
import open3d as o3d
import numpy as np
import glob
import os
import cv2
import math
import time

# --- Configuration ---
POINT_CLOUD_DIR = "/Users/hendrik/OMGrab/point_clouds"
OUTPUT_VIDEO_FILENAME = "point_cloud_video_centered_render.mp4" # New filename
OUTPUT_VIDEO_DIR = "generated_pointclouds"

# Resolution the point clouds were likely generated at
# Verify this matches the actual generation process
GENERATION_WIDTH = 1280
GENERATION_HEIGHT = 720

# Rendering resolution
VIS_WIDTH = 1920
VIS_HEIGHT = 1920
FPS = 15

# --- Intrinsics used during point cloud generation ---
# Assuming these correspond to GENERATION_WIDTH x GENERATION_HEIGHT
K_gen = np.array([
    [1.06933425e+03, 0.00000000e+00, 5.55076352e+02], # fx_gen, 0, cx_gen
    [0.00000000e+00, 1.06959363e+03, 9.45074567e+02], # 0, fy_gen, cy_gen
    [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]
])

# --- End Configuration ---

def create_video_from_point_clouds_centered_render( # Renamed function
    pcd_dir,
    output_dir,
    output_filename,
    render_width,
    render_height,
    fps,
    gen_intrinsics, # Still need fx, fy from generation
    gen_width,
    gen_height
):
    """
    Generates video using stereo_calibrate method, but forces the rendering
    principal point to the center of the output frame.
    """
    os.makedirs(output_dir, exist_ok=True)
    output_video_path = os.path.join(output_dir, output_filename)
    ply_files = sorted(glob.glob(os.path.join(pcd_dir, "frame_*.ply")))

    if not ply_files:
        print(f"Error: No .ply files found in {pcd_dir} matching 'frame_*.ply'.")
        return
    print(f"Found {len(ply_files)} point cloud files in {pcd_dir}.")
    print(f"Output video: {output_video_path} ({render_width}x{render_height} @ {fps}fps)")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (render_width, render_height))
    if not out_video.isOpened():
        print(f"Error: Could not open video writer for path {output_video_path}")
        return

    # --- Open3D Visualizer Setup ---
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=render_width, height=render_height, visible=False)
    render_options = vis.get_render_option()
    render_options.background_color = np.asarray([0.0, 0.0, 0.0])
    render_options.point_size = 1.0
    print("Open3D visualizer created.")

    # --- Define Point Cloud Rotation ---
    R_pc_fix = o3d.geometry.get_rotation_matrix_from_xyz((np.pi, 0, 0))
    print("Defined point cloud rotation matrix (180 deg around X).")

    # --- Calculate SCALED FOCAL LENGTH and CENTERED PRINCIPAL POINT ---
    scale_x = render_width / gen_width
    scale_y = render_height / gen_height

    # Scale focal lengths based on generation intrinsics and render size
    fx_render = gen_intrinsics[0, 0] * scale_x
    fy_render = gen_intrinsics[1, 1] * scale_y

    # FORCE principal point to the center of the rendering window
    cx_render = render_width / 2.0
    cy_render = render_height / 2.0
    print(f"Using Scaled Focal Lengths: fx={fx_render:.2f}, fy={fy_render:.2f}")
    print(f"Using Centered Principal Point: cx={cx_render:.2f}, cy={cy_render:.2f}")


    # Create the intrinsic object for the rendering camera
    intrinsic_render = o3d.camera.PinholeCameraIntrinsic(
        render_width, render_height, fx_render, fy_render, cx_render, cy_render
    )
    print("Created rendering intrinsic matrix:\n", intrinsic_render.intrinsic_matrix)


    # --- Set Camera View using Intrinsics and Fixed Extrinsic ---
    view_control = vis.get_view_control()
    pinhole_params = view_control.convert_to_pinhole_camera_parameters()

    # Set the calculated intrinsic parameters
    pinhole_params.intrinsic = intrinsic_render

    # Set the fixed extrinsic matrix (Camera at origin, looking down -Z after pcd rotation)
    # Keep small Z offset
    fixed_extrinsic = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, -1, 0.1], # Small Z offset
        [0, 0, 0, 1]
    ])
    pinhole_params.extrinsic = fixed_extrinsic
    print("Setting fixed camera extrinsic matrix:\n", fixed_extrinsic)

    # Apply the modified parameters back to the view control
    view_control.convert_from_pinhole_camera_parameters(pinhole_params, allow_arbitrary=True)

    # Adjust zoom level - start closer to 1.0 now that view should be centered
    zoom_level = 0.9 # Larger value = less zoom. Adjust if needed.
    view_control.set_zoom(zoom_level)
    print(f"Camera view parameters set. Zoom level: {zoom_level}")

    # ... (Initialize visualizer state with first frame - remains the same) ...
    try:
        pcd_initial = o3d.io.read_point_cloud(ply_files[0])
        pcd_initial.rotate(R_pc_fix, center=(0, 0, 0))
        vis.add_geometry(pcd_initial)
        vis.poll_events()
        vis.update_renderer()
        vis.clear_geometries()
        print("Initial visualizer state set.")
    except Exception as e:
        print(f"Warning: Could not load or process first point cloud for initial setup: {e}")


    # --- Loop through saved files and render ---
    print(f"Rendering {len(ply_files)} point clouds...")
    start_time = time.time()
    # ... (The loop structure remains the same) ...
    for i, ply_file in enumerate(ply_files):
        if (i+1) % 50 == 0:
             print(f"  Processing file {i+1}/{len(ply_files)}: {os.path.basename(ply_file)}")
        try:
            pcd = o3d.io.read_point_cloud(ply_file)
            if not pcd.has_points():
                print(f"  Warning: Skipping empty point cloud file {os.path.basename(ply_file)}.")
                frame_bgr = np.zeros((render_height, render_width, 3), dtype=np.uint8)
            else:
                # Apply the 180-degree X-axis rotation to the point cloud
                pcd.rotate(R_pc_fix, center=(0, 0, 0))

                vis.clear_geometries()
                vis.add_geometry(pcd)

                # Camera view is fixed by parameters set outside the loop
                vis.poll_events()
                vis.update_renderer()

                # Capture frame
                o3d_img = vis.capture_screen_float_buffer(do_render=True)
                frame_np = (np.asarray(o3d_img) * 255).astype(np.uint8)
                frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)

        except Exception as e:
            print(f"  Error reading or rendering file {ply_file}: {e}")
            frame_bgr = np.zeros((render_height, render_width, 3), dtype=np.uint8)

        out_video.write(frame_bgr)

    # --- Cleanup ---
    # ... (Cleanup remains the same) ...
    end_time = time.time()
    print(f"\nRendering complete in {end_time - start_time:.2f} seconds.")
    out_video.release()
    vis.destroy_window()
    cv2.destroyAllWindows()
    print(f"Output video saved to: {output_video_path}")


# --- Main Execution ---
if __name__ == "__main__":
    # Call the renamed function
    create_video_from_point_clouds_centered_render(
        pcd_dir=POINT_CLOUD_DIR,
        output_dir=OUTPUT_VIDEO_DIR,
        output_filename=OUTPUT_VIDEO_FILENAME,
        render_width=VIS_WIDTH,
        render_height=VIS_HEIGHT,
        fps=FPS,
        gen_intrinsics=K_gen,
        gen_width=GENERATION_WIDTH,
        gen_height=GENERATION_HEIGHT
    )

Found 429 point cloud files in /Users/hendrik/OMGrab/point_clouds.
Output video: generated_pointclouds/point_cloud_video_centered_render.mp4 (1920x1920 @ 15fps)
Open3D visualizer created.
Defined point cloud rotation matrix (180 deg around X).
Using Scaled Focal Lengths: fx=1604.00, fy=2852.25
Using Centered Principal Point: cx=960.00, cy=960.00
Created rendering intrinsic matrix:
 [[1.60400138e+03 0.00000000e+00 9.60000000e+02]
 [0.00000000e+00 2.85224968e+03 9.60000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Setting fixed camera extrinsic matrix:
 [[ 1.   0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.  -1.   0.1]
 [ 0.   0.   0.   1. ]]
Camera view parameters set. Zoom level: 0.9
Initial visualizer state set.
Rendering 429 point clouds...
  Processing file 50/429: frame_00049.ply
  Processing file 100/429: frame_00099.ply
  Processing file 150/429: frame_00149.ply
  Processing file 200/429: frame_00199.ply
  Processing file 250/429: frame_00249.ply
  Processing file 3